In [2]:
import numpy as np
from numpy.core.numeric import False_
import pandas as pd
from pandas.core.series import Series
import matplotlib.pyplot as plt 
from matplotlib import pyplot 

data = pd.read_csv("/Users/akanksha/Summer Project/archives/mtsdata_ucr/CoST/data_with_iteration.csv")

In [3]:
# Filter Out Variant 3 from the data.

data_var3 = data[' variant'] != 3
data = data[data_var3]

max_len = max(data[' frame'])
print('Max length is', max_len)

data = data.drop(' frame', axis = 1)

Max length is 1747


In [4]:
# # ---- STEP 3.a ------ divide train and test data ------

# #create training data
train = data['subject'] <= 21
train_data = data[train]
train_data = train_data[:1000]
# print('Train Data -')
# print(train_data.head)

test = data['subject'] > 21
test_data= data[test]
print('Original Length is ', test_data.shape)
test_data = test_data[:1000]
print('Test Length is ', test_data.shape)

Original Length is  (306593, 68)
Test Length is  (1000, 68)


In [9]:
# # # ---- STEP 4 ------ reshape X_train data into 3D data ------

# TRAIN DATA
exclusions = ['subject', ' variant', ' gesture', 'iteration']
print('X train data shape', train_data.shape)
cell_columns = [col for col in list(train_data.columns) if col not in exclusions]
x_train_data_3d = train_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[cell_columns].apply(lambda x : x.values.tolist()).tolist()

print('X Train data lengths', [len(x) for x in x_train_data_3d])

cell_columns = [col for col in list(train_data.columns) if col not in exclusions]
# train_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[' gesture'].apply(lambda x : print(x.values.tolist()[0]))
y_train_data_3d = train_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[' gesture'].apply(lambda x : x.values.tolist()[0]).tolist()


# print(len(train_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[' gesture'].apply(lambda x : x.values.tolist())[0]))

# TEST DATA
print('X test data shape', train_data.shape)
cell_columns = [col for col in list(test_data.columns) if col not in exclusions]
x_test_data_3d = test_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[cell_columns].apply(lambda x : x.values.tolist()).tolist()

print('X Test data lengths', [len(x) for x in x_test_data_3d])

cell_columns = [col for col in list(test_data.columns) if col not in exclusions]
y_test_data_3d = test_data.groupby(['subject', ' variant', ' gesture', 'iteration'])[' gesture'].apply(lambda x : x.values.tolist()[0]).tolist()
# print('y test data length is',len(y_train_data_3d), len(y_train_data_3d[0]))

X train data shape (1000, 68)
X Train data lengths [64, 67, 67, 88, 121, 88, 250, 255]
X test data shape (1000, 68)
X Test data lengths [211, 199, 127, 79, 91, 109, 142, 42]


In [12]:
# # ---- STEP 5 ------ Resample and interpolate to get equal lenght samples ------

# TRAIN DATA
x_train_interpolated = []
for video_index in range(len(x_train_data_3d)):
    channel = list(map(lambda x: x, x_train_data_3d[video_index]))
    # print(channel)
    channel_dataframe = pd.DataFrame(x_train_data_3d[video_index])
    index = pd.timedelta_range(start="1 days 00:00:00", end=str(max_len) + " days 00:00:00", periods=len(x_train_data_3d[video_index]))
    
    import datetime as datetime
    def round_to_1day(t):
        return t.round(freq='d')
    index = index.map(round_to_1day)
    channel_dataframe.set_index(index, inplace=True)
    interpolated = channel_dataframe.resample("d").interpolate(method="linear")
    # print(type(interpolated.values.tolist()))
    x_train_interpolated.append(interpolated.values.tolist())

print('X Train Interpolated data lengths', [len(x) for x in x_train_interpolated])

X Train Interpolated data lengths [1747]
